In [166]:
#Setting all the input variables

storm_name = "dorian"
storm_year = 2019
storm_lowerLat= 40

latExtension = 2
lonExtension = 2
dateExtension = 2

focus_variable = "wind_spd_avg"
var_units = "(m s-1)"

In [162]:
# Getting storm data and matching ERDDAP datasets around the storm (time and space)

import tropycal.tracks as tracks
import pandas as pd
from erddapy import ERDDAP
from datetime import timedelta

basin = tracks.TrackDataset(basin='north_atlantic', source='ibtracs')
storm = basin.get_storm((storm_name,storm_year))

# Coordinate selector throws an error when using with Fiona but still generates the graph
storm = storm.sel(lat=[storm_lowerLat,None])

storm_df = storm.to_dataframe()

min_lat = storm_df['lat'].min()
min_lon = storm_df['lon'].min()
max_lat = storm_df['lat'].max()
max_lon = storm_df['lon'].max()
start_date = storm_df.min()['date']
end_date = storm_df.max()['date']

# Get datasets matching time and location criteria, plus a buffer zone

e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv"
)

e.dataset_id = "allDatasets"

e.variables = [
    "datasetID",
    "minLongitude",
    "maxLongitude",
    "minLatitude",
    "maxLatitude",
    "minTime",
    "maxTime"
]

e.constraints = {
    "minTime<=": end_date + timedelta(days = dateExtension),
    "maxTime>=": start_date- timedelta(days = dateExtension),
    "minLatitude>=": min_lat - latExtension,
    "maxLatitude<=": max_lat + latExtension,
    "minLongitude>=": min_lon - lonExtension,
    "maxLongitude<=": max_lon + lonExtension,
}

datasets_df = e.to_pandas(
    parse_dates=True,
).dropna()
datasets = set(datasets_df["datasetID"])

--> Starting to read in ibtracs data
--> Completed reading in ibtracs data (10.02 seconds)
Using no lon bounds


In [167]:
# Narrow down datasets that contain the focused variable
# Some datasets may not match at certain points if their coverage is inconsistent

matching_datasets = []
for dataset in datasets:
    info_url = e.get_info_url(dataset_id = dataset, response = "csv")
    dataset_info = pd.read_csv(info_url)

    variable_names = set(dataset_info["Variable Name"].unique())
    if focus_variable in variable_names:
         matching_datasets.append(dataset)

print(matching_datasets)

['SMA_port_aux_basqes_wharf', 'SMA_halifax_fairview', 'sma_negl_red_bay_nlqu0005', 'SMA_halifax_anemometer1', 'SMA_saint_john_wharf', 'SMA_Fortune_Bay_Buoy', 'SMA_halifax', 'sma_negl_cartwright_junction_nlqu0004', 'SMA_Holyrood_Buoy2', 'SMA_MouthofPlacentiaBayBuoy', 'SMA_red_island_shoal', 'SMA_holyrood_wharf', 'SMA_saint_john', 'SMA_port_aux_basques', 'DFO_Sutron_KLUMI', 'cna_werc_weather_10-min_avg', 'sma_negl_black_tickle_nlqu0003', 'SMA_saint_john_cruise_terminal', 'SMA_halifax_pier9c', 'sma_negl_north_west_river_nlqu0007']


In [173]:
# Find most recent values for focus var given a time point

selected_time=  "2019-09-07 17:30:00"

e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv",
)

for dataset in matching_datasets:

    e.dataset_id = dataset
    print(dataset)
    
    e.constraints = {
        "time<=": selected_time
    }
        
    e.variables = ["time", "longitude", "latitude", focus_variable]

    try:
        buoy_data= e.to_pandas(
            parse_dates=True,
        ).dropna()
        print(buoy_data.tail(1).iloc[0]["time (UTC)"])
        print(buoy_data.tail(1).iloc[0][focus_variable + " " + var_units])
    except:
        print("couldn't find info for the given dataset")
    print("-")


2019-09-07 17:30:00
SMA_port_aux_basqes_wharf
2019-09-07T17:29:52Z
10.0
-
SMA_halifax_fairview
2019-09-07T17:30:00Z
7.6
-
sma_negl_red_bay_nlqu0005
2019-09-07T16:52:00Z
5.9166665
-
SMA_halifax_anemometer1
2019-09-07T17:30:00Z
22.3
-
SMA_saint_john_wharf
2019-09-07T17:29:36Z
16.3
-
SMA_Fortune_Bay_Buoy
2019-09-07T17:23:41Z
8.4
-
SMA_halifax
2019-09-07T17:23:01Z
21.9
-
sma_negl_cartwright_junction_nlqu0004
2019-09-07T16:52:00Z
1.7777778
-
SMA_Holyrood_Buoy2
2019-09-07T17:23:00Z
7.2
-
SMA_MouthofPlacentiaBayBuoy
2019-04-30T14:23:01Z
5.4
-
SMA_red_island_shoal
2019-09-07T17:25:00Z
7.4
-
SMA_holyrood_wharf
2019-09-07T17:14:55Z
3.9
-
SMA_saint_john
2019-09-07T17:23:01Z
13.5
-
SMA_port_aux_basques
2019-09-07T17:24:01Z
12.5
-
DFO_Sutron_KLUMI
2019-04-16T21:10:00Z
0.01
-
cna_werc_weather_10-min_avg
2019-09-07T17:30:00Z
couldn't find info for the given dataset
-
sma_negl_black_tickle_nlqu0003
2019-09-07T16:52:00Z
3.3055556
-
SMA_saint_john_cruise_terminal
2019-09-07T17:30:00Z
5.1
-
SMA_halifax_p

In [ ]:
# Create plots  for matching datasets
e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv",
)

for dataset in matching_datasets:

    # Still keeps generating graph after dataset fails for some reason?

    e.dataset_id = dataset

    e.constraints = {
        "time>=": start_date - timedelta(days = 2),
        "time<=": end_date + timedelta(days = 2)
    }

    e.variables = ["time", "longitude", "latitude", focus_variable]

    try:
        buoy_data= e.to_pandas(
            parse_dates=True,
        ).dropna()
        buoy_data.plot(x='time (UTC)', y=focus_variable + ' ' + var_units, title=dataset)
    except:
        print("Data does not exist for %s", dataset) 
